In [5]:
from kubernetes import client
from kfserving import KFServingClient
from kfserving import constants
from kfserving import utils
from kfserving import V1alpha2EndpointSpec
from kfserving import V1alpha2PredictorSpec
from kfserving import V1alpha2TensorflowSpec
from kfserving import V1alpha2InferenceServiceSpec
from kfserving import V1alpha2InferenceService
from kubernetes.client import V1ResourceRequirements
import os
import sys
import argparse
import logging

'''
'''
class KFServingOnNoteBook(object):
    def serving(self, storage_uri, namespace, serving_name):
        api_version = constants.KFSERVING_GROUP + '/' + constants.KFSERVING_VERSION
        default_endpoint_spec = V1alpha2EndpointSpec(
                                  predictor=V1alpha2PredictorSpec(
                                    tensorflow=V1alpha2TensorflowSpec(
                                      storage_uri=storage_uri,
                                      resources=V1ResourceRequirements(
                                          requests={'cpu':'100m','memory':'1Gi'},
                                          limits={'cpu':'100m', 'memory':'1Gi'}))))
        isvc = V1alpha2InferenceService(api_version=api_version,
                                  kind=constants.KFSERVING_KIND,
                                  metadata=client.V1ObjectMeta(
                                      name=serving_name, namespace=namespace),
                                  spec=V1alpha2InferenceServiceSpec(default=default_endpoint_spec))        

        KFServing = KFServingClient()
        KFServing.create(isvc)

        KFServing.get(serving_name, namespace=namespace, watch=True, timeout_seconds=300)
        
if __name__ == '__main__':
    serving_service = KFServingOnNoteBook()
    serving_service.serving("pvc://azurefile/saved_model", "kf-namespace", "fashion-mnist-server")
        

RuntimeError: Exception when calling CustomObjectsApi->create_namespaced_custom_object:                 (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'Audit-Id': '9dc561a9-4622-4ab0-9e88-4fe153d4e960', 'Cache-Control': 'no-cache, private', 'Content-Type': 'application/json', 'Date': 'Wed, 12 Aug 2020 07:29:32 GMT', 'Content-Length': '296'})
HTTP response body: {"kind":"Status","apiVersion":"v1","metadata":{},"status":"Failure","message":"inferenceservices.serving.kubeflow.org \"fashion-mnist-server\" already exists","reason":"AlreadyExists","details":{"name":"fashion-mnist-server","group":"serving.kubeflow.org","kind":"inferenceservices"},"code":409}




In [2]:
!kubectl get inferenceservice

NAME                   URL                                                                                   READY   DEFAULT TRAFFIC   CANARY TRAFFIC   AGE
fashion-mnist-server   http://fashion-mnist-server.kf-namespace.example.com/v1/models/fashion-mnist-server   True    100                                55s


In [3]:
!kubectl get svc

NAME                                                   TYPE           CLUSTER-IP     EXTERNAL-IP                                            PORT(S)                             AGE
fashion-mnist-server-predictor-default                 ExternalName   <none>         cluster-local-gateway.istio-system.svc.cluster.local   <none>                              109s
fashion-mnist-server-predictor-default-np958           ClusterIP      10.0.24.98     <none>                                                 80/TCP                              2m8s
fashion-mnist-server-predictor-default-np958-private   ClusterIP      10.0.114.154   <none>                                                 80/TCP,9090/TCP,9091/TCP,8022/TCP   2m8s
lab                                                    ClusterIP      10.0.249.240   <none>                                                 80/TCP                              2d1h


In [4]:
!saved_model_cli show --dir ./saved_model/001 --tag_set serve --signature_def serving_default

2020-08-12 07:01:57.483327: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2020-08-12 07:01:57.483381: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
The given SavedModel SignatureDef contains the following input(s):
  inputs['flatten_input'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 28, 28)
      name: serving_default_flatten_input:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['dense_1'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 10)
      name: StatefulPartitionedCall:0
Method name is: tensorflow/serving/predict
